In [2]:
import cv2
import os
from pathlib import Path

In [3]:
video_path = "data/kagome_raw/ketchup/20250424_I_3kg_2.avi"
output_dir = "data/kagome_raw/ketchup/train"

In [4]:
def save_avi_frames_as_jpeg(video_path: str, output_dir: str) -> None:
    """
    AVI 形式の動画ファイルから全フレームを JPG で保存する関数。

    Parameters
    ----------
    video_path : str
        入力する AVI ファイルのパス。
    output_dir : str
        フレームを保存する出力フォルダのパス。存在しない場合は自動で作成する。

    Raises
    ------
    FileNotFoundError
        video_path が存在しない場合。
    RuntimeError
        動画の読み込みに失敗した場合。
    """
    video_path = Path(video_path)
    output_dir = Path(output_dir)

    # 1) 入力ファイルの存在確認
    if not video_path.is_file():
        raise FileNotFoundError(f"入力ファイルが見つかりません: {video_path}")

    # 2) 出力フォルダを作成（すでに存在していても OK）
    output_dir.mkdir(parents=True, exist_ok=True)

    # 3) OpenCV で動画を開く
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"動画ファイルを開けませんでした: {video_path}")

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:                 # 末尾まで読んだら終了
            break

        # 4) JPEG で保存
        frame_filename = output_dir / f"frame_{frame_idx:06d}.jpg"
        cv2.imwrite(str(frame_filename), frame)

        frame_idx += 1

    cap.release()
    print(f"完了: {frame_idx} 枚のフレームを {output_dir} に保存しました。")


In [5]:
save_avi_frames_as_jpeg(video_path,output_dir)

完了: 1000 枚のフレームを data/kagome_raw/ketchup/train に保存しました。


In [7]:
def prune_jpegs_every_nth(output_dir: str, n: int = 50) -> None:
    """
    すでに保存された JPEG 画像を、n 枚ごと（デフォルト 50 枚ごと）に 1 枚だけ残し、
    それ以外の JPEG を削除する関数。

    Parameters
    ----------
    output_dir : str
        JPEG ファイルが保存されているフォルダ。
    n : int, optional
        何枚おきに 1 枚を残すか（既定値は 50）。

    Raises
    ------
    FileNotFoundError
        output_dir が存在しない場合。
    """
    output_dir = Path(output_dir)

    # 1) 出力フォルダの存在確認
    if not output_dir.is_dir():
        raise FileNotFoundError(f"出力フォルダが見つかりません: {output_dir}")

    # 2) JPEG ファイルを昇順で取得（frame_000001.jpg, frame_000002.jpg, ...）
    jpg_files = sorted(output_dir.glob("*.jpg"))

    if not jpg_files:
        print("JPEG ファイルが見つかりませんでした。")
        return

    removed, kept = 0, 0
    for idx, jpg_path in enumerate(jpg_files):
        # n 枚おきに 1 枚残し、それ以外を削除
        if idx % n == 0:
            kept += 1
            continue
        try:
            jpg_path.unlink()   # ファイル削除
            removed += 1
        except Exception as e:
            print(f"削除失敗: {jpg_path} ({e})")

    print(f"{kept} 枚を残し、{removed} 枚を削除しました。")

In [8]:
prune_jpegs_every_nth(output_dir, n=50)

20 枚を残し、980 枚を削除しました。
